Подключение библиотек

In [33]:
from pymongo import MongoClient
from pprint import pprint

Далее, создаем клиента для подключения к серверу. Сервер в нашем примере запущен локально без изменений параметров по умолчанию:

In [12]:
client = MongoClient('localhost', 27017)

Один клиент может работать с несколькими независимыми базами данных. Когда работаем с PyMongo, нам необходимо подключиться к базе данных, указав ее имя:

In [13]:
db = client['test_database']

В рамках одной базы данных мы можем работать с несколькими коллекциями. Создадим указатель на коллекцию, чтобы проще было обращаться к ней:
collection = db.test_collection

In [2]:
collection = db.test_collection

## Операции CRUD
CRUD операции - это создание (Create), чтение (Read), обновление (Update), Удаление (Delete)

### Операции Create
Операции Create (или их иногда называют Insert) добавляют новый документ в коллекцию. MongoDB предоставляет следующие методы для того, чтобы добавить новые документы в коллекцию:
```python
db.collection.insertOne() 
db.collection.insertMany()
```

Особенности при создании/добавлении нового документа:
1. Если коллекция, указанная при создании документа не существует, то она создается.
2. В MongoDB, каждый документ, записываемый в коллекцию должен иметь поле _id с уникальным значением, которое выступает как первичный ключ. Если добавляемый документ не имеет данного поля, MongoDb автоматически сгенерирует ObjectId для данного поля _id
3. В MongoDB операции insert всегда работают только с одной коллекцией. Нельзя вставить документ сразу в несколько коллекций, используя метод один раз. Также важно понимать, что все операции записи в БД атомарны на уровне одного документа

#### `db.collection.insertOne()` добавляет один документ в коллекцию, переданный в качестве параметра метода:

In [17]:
collection.insert_one(
   { 'item': "canvas", 
     'qty': 100, 
     'tags': ["cotton"], 
     'size': { 'h': 28, 'w': 35.5, 'uom': "cm" } })

#### `db.collection.insertMany()` добавляет несколько документов в коллекцию. Для этого необходимо передать список документов в качестве параметра метода:

In [19]:
collection.insert_many([
   { 'item': "journal", 
    'qty': 25, 'tags': ["blank", "red"], 
    'size': { 'h': 14, 'w': 21, 'uom': "cm" } },
   { 'item': "mat", 
    'qty': 85, 
    'tags': ["gray"], 
    'size': { 'h': 27.9, 'w': 35.5, 'uom': "cm" } },
   { 'item': "mousepad", 
    'qty': 25, 
    'tags': ["gel", "blue"], 
    'size': { 'h': 19, 'w': 22.85, 'uom': "cm" } }
])

В указанном выше примере ни один из документов также не имеет поля `_id`. В этом случае MongoDB добавляет его к каждому документу из списка. Метод `insertMany()` также возвращает документ, который содержит в себе все новые добавленные документы с полями `_id`

### Операции Read
Операции Read запрашивают документы из коллекции по указанным критериям. MongoDB предоставляет всего один метод для того, чтобы получить необходимые документы:
```python
db.collection.find()
```

Вы можете также изменять результат работы метода с помощью критериев, проекций, и дополнительных методов - модификаторов.

#### Запрос всех документов в коллекции:
Чтобы получить все документы в коллекции, передайте пустой документ в качестве параметра метода `.find()`

In [30]:
collection.find( {} )

In [34]:
for i in collection.find( {} ):
    pprint(i)

{'_id': ObjectId('5e6e0683b32988511d83f7c4'),
 'item': 'canvas',
 'qty': 100,
 'size': {'h': 28, 'uom': 'cm', 'w': 35.5},
 'tags': ['cotton']}
{'_id': ObjectId('5e6e084cb32988511d83f7c6'),
 'item': 'canvas',
 'qty': 100,
 'size': {'h': 28, 'uom': 'cm', 'w': 35.5},
 'tags': ['cotton']}
{'_id': ObjectId('5e6e0858b32988511d83f7c7'),
 'item': 'journal',
 'qty': 25,
 'size': {'h': 14, 'uom': 'cm', 'w': 21},
 'tags': ['blank', 'red']}
{'_id': ObjectId('5e6e0858b32988511d83f7c8'),
 'item': 'mat',
 'qty': 85,
 'size': {'h': 27.9, 'uom': 'cm', 'w': 35.5},
 'tags': ['gray']}
{'_id': ObjectId('5e6e0858b32988511d83f7c9'),
 'item': 'mousepad',
 'qty': 25,
 'size': {'h': 19, 'uom': 'cm', 'w': 22.85},
 'tags': ['gel', 'blue']}


Данная операция аналогична запросу в реляционной СУБД:
```SQL
SELECT * FROM collection
```

#### Запрос документов по заданному условию:
Чтобы получить результат по определенному критерию поиска, необходимо передавать пары `{<field1>: <value1>, ... }` в качестве параметра метода `.find()`. 

Так, следующий пример вернет все документы, значение поля `“status”` которых равны `“D”`:

In [26]:
collection.find( { 'status': "D" } )

In [56]:
collection.find({})

In [36]:
for i in collection.find( { 'qty': 25 } ):
    pprint(i)

{'_id': ObjectId('5e6e0858b32988511d83f7c7'),
 'item': 'journal',
 'qty': 25,
 'size': {'h': 14, 'uom': 'cm', 'w': 21},
 'tags': ['blank', 'red']}
{'_id': ObjectId('5e6e0858b32988511d83f7c9'),
 'item': 'mousepad',
 'qty': 25,
 'size': {'h': 19, 'uom': 'cm', 'w': 22.85},
 'tags': ['gel', 'blue']}


Данная операция аналогична запросу в реляционной СУБД:
```SQL
SELECT * FROM inventory WHERE 'qty' = 25
```

#### Запрос документов по нескольким критериям:
Для того, чтобы усложнить условия выборки из документов, используют специальные операторы запросов:
1. Операторы сравнения: `$eq, $gt, $gte, $in, $lt, $lte, $ne, $nin`
2. Логические операторы: `$and, $not, $nor, $or`
3. Операторы элементов: `$exists, $type`
4. Операторы оценки: `$expr, $jsonSchema, $mod, $regex, $text, $where`
Здесь перечислены основные. Про дополнительные можно прочитать в документации к MongoDB. 

Синтаксис применения операторов:
```python
{ <field1>: { <operator1>: <value1> }, ... }
```

Следующий пример запрашивает все документы из коллекции inventory  у которых поле  status равно “A” или  “D”:

In [52]:
for qty in db.inventory.find( { 'qty': { '$in': [ 25, 100] } } ):
    print(qty)

**Важно!**
Данный запрос можно составить с применением оператора `‘$or’`. Результат будет тот же самый. Но когда мы проверяем значения в одном и том же поле у всех документов, то рекомендуется использовать оператор` $’in’` как более быстрый и правильный при решении данной задачи.

Данная операция аналогична запросу в реляционной СУБД:
```SQL
SELECT * FROM inventory WHERE status in ("A", "D")
```

#### Запрос документов по составным условиям (оператор `$and`):
Вы можете выполнить составной запрос, используя критерии выборки по двум и более полям документов. 

Обычно для этого используется оператор `$and` (конъюнкция или логическое И) и дальше в списке указывается две и более пар <поле>:<значение>. Пример ниже запрашивает все документы в коллекции inventory со значениями status = “A” и qty меньше 30:
db.inventory.find( { $and: [ { status: "A" }, { qty: { $lt: 30 } } ] } )
 
Но, по умолчанию, данный оператор будет применяться ко всем парам, если их просто указать в качестве параметра метода .find():
db.inventory.find( { status: "A", qty: { $lt: 30 } } )
 
Результат выполнения обоих выражений будет одинаковый.
Данная операция аналогична запросу в реляционной СУБД:
SELECT * FROM inventory WHERE status = "A" AND qty < 30
 
Запрос документов по составным условиям (оператор $or):
Использование оператора $or (дизъюнкция или логическое “ИЛИ”) позволяет объединять результаты запроса по каждой паре <поле>:<значение> из списка, переданного в качестве параметра метода .find(). Для того, чтобы запрос сработал, необходимо по крайней мере наличие одного условия. Пример ниже запрашивает все документы в коллекции inventory со значениями status = “A” или qty меньше 30:
db.inventory.find( { $or: [ { status: "A" }, { qty: { $lt: 30 } } ] } )
 
Данная операция аналогична запросу в реляционной СУБД:
SELECT * FROM inventory WHERE status = "A" OR qty < 30

Важно! 
Запросы, в которых используются операторы сравнения работают корректно только в том случае если тип сравниваемого значения и тип значения поля документа одинаковы!

Запрос документов по составным условиям (совместное использование $or и $and):
В следующем примере составной запрос возвращает все документы в коллекции, у которых значение поля status равно “А” и также qty меньше ($lt) 30 или item начинается с символа “p”:
db.inventory.find( {
     status: "A",
     $or: [ { qty: { $lt: 30 } }, { item: /^p/ } ]
} )
 
Данная операция аналогична запросу в реляционной СУБД:
SELECT * FROM inventory WHERE status = "A" AND ( qty < 30 OR item LIKE "p%")

Проекции
Проекция - это метод, который определяет какие поля вернутся в документах, соответствующие критериям поиска. В качестве параметра передается документ вида:
{ field1: <value>, field2: <value> ... }

Значение <value> может быть следующим:
1 или true, чтобы включить поле в возвращаемый документ
0 или false, чтобы исключить поле из возвращаемого документа
Выражение, использующее операторы проекции: $, $elemMatch, $slice, $meta

Важно! 
Для поля _id не обязательно использовать проекцию _id: 1, чтобы включить поле _id. Метод .find() всегда будет возвращать данное поле, пока вы не укажете принудительно в проекции _id: 0, чтобы его убрать в выводе.

Пример ниже запрашивает все документы в коллекции inventory со значениями status = “A” и qty меньше 30. Но выводит только поля item и status и _id (выводится по умолчанию):
db.inventory.find( { status: "A", qty: { $lt: 30 } }, {item: 1, status: 1} )
Данная операция аналогична запросу в реляционной СУБД:
SELECT item, status FROM inventory WHERE status = "A" AND qty < 30
